# 🛠️ Fixed Notebook
This copy adds a robust setup cell (installs/imports packages if missing) and makes Google Drive mounting optional.
You can run it both in **Google Colab** and **local Jupyter** without manual edits.

In [ ]:

# === Robust environment setup (works in Colab and local Jupyter) ===
import sys, subprocess, importlib

def ensure(pkg, import_name=None):
    name = import_name or pkg
    try:
        importlib.import_module(name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
        importlib.invalidate_caches()

# Core deps
for pkg in ["numpy", "pandas", "matplotlib", "tensorflow", "opendatasets", "scikit-learn", "gradio"]:
    ensure(pkg)

# Common imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow / Keras imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Opendatasets
import opendatasets as od

print("✅ Environment ready. TensorFlow version:", tf.__version__)


In [ ]:

# === Optional: Mount Google Drive if running in Colab ===
try:
    import google.colab  # type: ignore
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted.")
except Exception as e:
    print("ℹ️ Not running in Google Colab (or drive mount skipped).")


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od

In [ ]:
# download dataset
od.download("https://www.kaggle.com/datasets/nafin59/monkeypox-skin-lesion-dataset/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: a
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nafin59/monkeypox-skin-lesion-dataset


100%|██████████| 46.3M/46.3M [00:00<00:00, 1.03GB/s]

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Parameters
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 1

In [ ]:
# Paths
train_dir = "/content/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Train"
val_dir = "/content/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val"


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, rotation_range=20)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2142 images belonging to 2 classes.
Found 420 images belonging to 2 classes.


In [ ]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(*IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train model
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen
)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.7219 - loss: 0.5319 - val_accuracy: 0.6524 - val_loss: 0.5896
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.7400 - loss: 0.5058 - val_accuracy: 0.6048 - val_loss: 0.6100
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.7380 - loss: 0.5029 - val_accuracy: 0.6857 - val_loss: 0.6191
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.7298 - loss: 0.5093 - val_accuracy: 0.5571 - val_loss: 0.6287
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.7476 - loss: 0.4986 - val_accuracy: 0.7048 - val_loss: 0.5807
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.7592 - loss: 0.4833 - val_accuracy: 0.7310 - val_loss: 0.5839
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.7556 - loss: 0.4896 - val_accuracy: 0.7071 - val_loss: 0.5917
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.7883 - loss: 0.4471 - val_accuracy: 0.6905 - val_loss

In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

# Assuming your model is already loaded as `model`
# If not, load like: model = load_model("monkeypox_detector.h5")

def predict_monkeypox(img):
    img = img.resize((128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]

    if prediction > 0.5:
        label = "Monkeypox"
        confidence = prediction
        meds = """
🧪 Antiviral Medications:
- Tecovirimat (TPOXX)
- Brincidofovir
- Cidofovir

🩺 Supportive Medications:
- Paracetamol (Acetaminophen) – for fever and pain
- Ibuprofen – for inflammation and pain
- Antihistamines (e.g., Cetirizine, Loratadine) – for itching
- Topical antiseptics (e.g., Chlorhexidine, Povidone-iodine) – for lesion care
"""
        return f"Prediction: {label} (Confidence: {confidence:.2f})\n\nRecommended Medicines:\n{meds}"
    else:
        label = "Non-Monkeypox"
        confidence = 1 - prediction
        return f"Prediction: {label} (Confidence: {confidence:.2f})"

interface = gr.Interface(
    fn=predict_monkeypox,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Monkeypox Detection with Medicine Recommendations",
    description="Upload an image to check for Monkeypox. If detected, recommended medicines will be displayed."
)

interface.launch(share=True)
